In [171]:
from sgp4.api import SGP4_ERRORS

from astropy.coordinates import TEME, CartesianRepresentation  # CartesianDifferential
from astropy import units as u
from astropy.coordinates import ITRS

import pymap3d as pm

import pandas as pd
from sgp4 import omm
from sgp4.api import Satrec
from astropy.time import Time
from astropy.time import TimeDelta
import time
import numpy as np

class PropagInit:
    def __init__(self, satellite, lc, sample_time=1):
        self.satellite = satellite
        self.lc = lc
        self.sample_time = sample_time

        self.tempo = []
        self.traj = []
        self.dist = []
        self.hdmin = []
        self.dmin = []
        self.hrmin = []

    def searchh0(self, t_inic, t_final, distmin, distmin2,  n):

        enu_dv = []
        tv = []
        enu_pv = []
        flagmin = True
        ntraj = 0

        tvar = t_inic

        hr = 0
        hrmin = 0

        for i in range(0, n):
            error_code, teme_p, teme_v = self.satellite.sgp4(tvar.jd1, tvar.jd2)
            if error_code != 0:
                # raise RuntimeError(SGP4_ERRORS[error_code])
                print(" SGP4_ERRORS: {}\n".format(SGP4_ERRORS[error_code]))
                break
            teme_p = CartesianRepresentation(teme_p * u.km)
            teme = TEME(teme_p, obstime=tvar)  # ver diferença entre UT1 e UTC
            itrsp = teme.transform_to(ITRS(obstime=tvar))
            location = itrsp.earth_location
            # location.geodetic
            enu_p = pm.ecef2enu(1000 * location.x.value, 1000 * location.y.value, 1000 * location.z.value, self.lc.lat,
                                self.lc.lon, self.lc.height)
            enu_d = (enu_p[0]**2 + enu_p[1]**2 + enu_p[2]**2)**0.5

            if enu_d > distmin:
                tsample = round(1+abs((enu_d - distmin)/10000))
                print("sample = {}".format(tsample))
                if not flagmin:
                    flagmin = True
                    self.tempo.append(tv)
                    self.traj.append(enu_pv)
                    self.dist.append(enu_dv)
                    self.hdmin.append(hdmin)
                    self.dmin.append(dmin)
                    self.hrmin.append(hrmin)
                    tv = []
                    enu_pv = []
                    enu_dv = []
                hr = 0
                hrmin = 0
            else:
                tsample = self.sample_time
                tv.append(tvar)
                enu_dv.append(enu_d)
                enu_pv.append(enu_p)
                # print('{} {:.3f}'.format(tvar.value, enu_d))
                if (enu_dv[len(enu_dv)-2] < enu_dv[len(enu_dv)-1]) and flagmin:
                    dmin = enu_dv[len(enu_dv) - 2]
                    if dmin < distmin2:
                        hdmin = tvar - TimeDelta(tsample * u.s)
                        hrmin = hr - 1
                        flagmin = False
                        ntraj += 1
                        print('NORAD: {}, H0: {}, dmin= {:.3f}'.format(self.satellite.satnum, tv[0].value, dmin))
                    else:
                        tsample = 60*5

            hr += 1
            tvar += TimeDelta(tsample * u.s) # tsample  / (60 * 60 * 24)

            if tvar > t_final: # Time.strptime('2021-8-25 19:00:00', '%Y-%m-%d %H:%M:%S'):
                print("fim loop")
                break
        return self

    def searchh01(self, t_inic, t_final, distmin, distmin2,  n):

        enu_dv = []
        tv = []
        enu_pv = []
        flagmin = True
        ntraj = 0

        tvar = t_inic

        hr = 0
        hrmin = 0

        for i in range(0, n):
            error_code, teme_p, teme_v = self.satellite.sgp4(tvar.jd1, tvar.jd2)
            if error_code != 0:
                # raise RuntimeError(SGP4_ERRORS[error_code])
                print(" SGP4_ERRORS: {}\n".format(SGP4_ERRORS[error_code]))
                break
            teme_p = CartesianRepresentation(teme_p * u.km)
            teme = TEME(teme_p, obstime=tvar)  # ver diferença entre UT1 e UTC
            itrsp = teme.transform_to(ITRS(obstime=tvar))
            location = itrsp.earth_location
            # location.geodetic
            enu_p = pm.ecef2enu(1000 * location.x.value, 1000 * location.y.value, 1000 * location.z.value, self.lc.lat,
                                self.lc.lon, self.lc.height)
            enu_d = (enu_p[0]**2 + enu_p[1]**2 + enu_p[2]**2)**0.5

            # if enu_d < distmin:
            #     print('ok')
            if (enu_d - distmin)>0:
                tsample = round(5+abs(1*(enu_d - distmin)/8000))
            else:
                tsample = round(5+abs(2*(enu_d - distmin)/8000))            

            print("sample = {}, dist = {}".format(tsample,enu_d ))

            tvar += TimeDelta(tsample * u.s) # tsample  / (60 * 60 * 24)

            if tvar > t_final: # Time.strptime('2021-8-25 19:00:00', '%Y-%m-%d %H:%M:%S'):
                print("fim loop")
                break
        return self

class LocalFrame:
    def __init__(self, lat, lon, height):
        self.lat = lat
        self.lon = lon
        self.height = height

In [174]:


orbital_elem = pd.read_csv("orbital_elem.csv").to_dict('records')
lc = LocalFrame(-5.923654, -35.167755, 50)

satellite = Satrec()
omm.initialize(satellite, orbital_elem[20])

posit = PropagInit(satellite,lc)


t_inic=Time('2022-08-10T11:00:00.000', format='isot', scale='utc')     # momento de inicio da busca
t_final= Time('2022-08-12T19:00:00.000', format='isot', scale='utc')   # momento de parar a busca

ini = time.time()
posit.searchh01(t_inic, t_final, 1100*1000,  1000*1000,  10000)
fim = time.time()
print("Tempo: ", fim - ini)

sample = 658, dist = 6324248.653669038
sample = 1093, dist = 9801265.566142354
sample = 1524, dist = 13249662.404228577
sample = 1257, dist = 11115179.23342537
sample = 380, dist = 4097507.5662682266
sample = 68, dist = 1603691.3219074856
sample = 20, dist = 1219115.0003244071
sample = 8, dist = 1120238.863648816
sample = 9, dist = 1083355.8504551298
sample = 19, dist = 1043991.2763392444
sample = 38, dist = 969487.5252119615
sample = 64, dist = 865490.1355757797
sample = 62, dist = 870172.6776755357
sample = 11, dist = 1076581.3716648892
sample = 8, dist = 1127248.3935209163
sample = 13, dist = 1165983.0246351906
sample = 21, dist = 1231878.8845756068
sample = 36, dist = 1344780.1760657642
sample = 62, dist = 1552163.6215770424
sample = 109, dist = 1934842.171621607
sample = 198, dist = 2644147.899406283
sample = 363, dist = 3961129.3904729104
sample = 656, dist = 6304167.240899046
sample = 1110, dist = 9938533.920569584
sample = 1528, dist = 13286954.974468008
sample = 1228, dist = 1

In [ ]:
posit.tempo

1.0

In [79]:
from skyfield.api import load, wgs84

tles1 = load.tle_file('conftle.txt')

satellite = tles1[5]
print(satellite)

ts = load.timescale()

bluffton = wgs84.latlon(-5.92256, -35.1615)
t0 = ts.utc(2021, 11, 26)
t1 = ts.utc(2021, 11, 27)
t, events = satellite.find_events(bluffton, t0, t1, altitude_degrees=10.0)
for ti, event in zip(t, events):
    name = ('rise above 10°', 'culminate', 'set below 10°')[event]
    print(ti.utc_strftime('%Y %b %d %H:%M:%S'), name)

0.0